In [5]:
import pandas as pd
from functools import reduce
import random

In [6]:
file_paths = ['../data/cleanedData/oxygenAB.parquet',
              '../data/cleanedData/nitrate.parquet',
              '../data/cleanedData/phosphate.parquet',
              '../data/cleanedData/ammonium.parquet',
              '../data/shiftedDates/total.parquet']

dfs = [pd.read_parquet(path) for path in file_paths]

merged = reduce(lambda left, right: pd.merge(left, right, how='inner', on='measurementDate'), dfs)
# 1.4 is the std of the phosphate
phosphate_average = merged['phosphateValue'].mean() 
merged['phosphateValue'] = merged['phosphateValue'].map(lambda phosphate: phosphate_average + random.uniform(1.4,-1.4) if pd.isnull(phosphate) else phosphate)

In [7]:
rain = pd.read_csv("../data/cleanedData/precipitation.csv")

In [8]:
rain = rain.rename(columns={'Date': 'measurementDate'})
rain.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       366 non-null    int64  
 1   measurementDate  366 non-null    object 
 2   Rain (mm)        366 non-null    float64
dtypes: float64(1), int64(1), object(1)
memory usage: 8.7+ KB


In [9]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 526307 entries, 0 to 526306
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   oxygenValue         526307 non-null  float64       
 1   measurementDate     526307 non-null  datetime64[ns]
 2   nitrateValue        526307 non-null  float64       
 3   phosphateValue      526307 non-null  float64       
 4   ammoniumValue       526307 non-null  float64       
 5   waterFlowPerMinute  526286 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 28.1 MB


In [10]:
rain["measurementDate"].dtype

dtype('O')

In [11]:
# rain = rain.astype({"measurementDate": "datetime64[ns]"})
# date_range = pd.date_range(start=rain["measurementDate"].min(), end=rain["measurementDate"].max(), freq="T")
# minute_df = pd.DataFrame({"measurementDate": date_range})  

# def assign_precipitation(row):
#     date = row["measurementDate"].date()
#     daily_row = rain.loc[rain["measurementDate"].dt.date == date]
#     if not daily_row.empty:
#         return daily_row["Rain (mm)"].values[0]
#     else:
#         return 0.0

# minute_df["precipitation"] = minute_df.apply(assign_precipitation, axis=1)
# precip_resampled = rain.set_index("measurementDate")["Rain (mm)"].resample('T').ffill()
# precip_resampled_df = precip_resampled.to_frame("rain_per_minute").reset_index()  
# merged_df = minute_df.merge(precip_resampled_df, how="left", on="measurementDate")
# merged = merged.merge(merged_df[['measurementDate', 'precipitation']], how='left', on='measurementDate')


In [12]:
rain["measurementDate"] = pd.to_datetime(rain["measurementDate"])
minute_df = pd.DataFrame({"measurementDate": pd.date_range(start=rain["measurementDate"].min(), end=rain["measurementDate"].max(), freq="T")})

daily_precipitation = rain.groupby(rain['measurementDate'].dt.date)['Rain (mm)'].first().to_dict()

def assign_precipitation(row):
    date = row["measurementDate"].date()
    return daily_precipitation.get(date, 0.0)

minute_df["precipitation"] = minute_df.apply(assign_precipitation, axis=1)
precip_resampled = rain.set_index("measurementDate")["Rain (mm)"].resample('T').ffill().reset_index()
merged_df = pd.merge(minute_df, precip_resampled, how="left", on="measurementDate")
merged = pd.merge(merged, merged_df[['measurementDate', 'precipitation']], how='left', on='measurementDate')

In [13]:
merged.to_parquet('../data/cleanedData/allData.parquet')